# Iodine Spectrometer Data Collection

In [ ]:
import time
from time import perf_counter, sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydaqmx_helper.adc import ADC

### Variable Naming

In [ ]:
sample_rate = 5000  # samples per second
bin_size = 0.05  # seconds per bin
countdown = 5  # seconds to wait before starting acquisition

In [ ]:
stop = False
def _enter_to_stop():
    global stop
    input("Press Enter to stop...\n")
    stop = True

In [ ]:
for s in range(countdown, 0, -1): # counting down so i know when to start the motor
    print(s)
    sleep(1)
print("Starting acquisition...")


In [ ]:
loop_stop = 0

myADC = ADC()
myADC.addChannels([0], minRange = -10, maxRange = 10)
data = myADC.sampleVoltages(sample_rate * bin_si

while loop_stop == 0:



In [ ]:
import time
from threading import Event, Thread
from pathlib import Path
import numpy as np
import pandas as pd
from pydaqmx_helper.adc import ADC  


sample_rate = 5000  
samples_num = 500

def _wait_for_enter(stop_event: Event):
    input("Press ENTER to stop acquisition...\n")
    stop_event.set()

def acquire_channel0_simple(sample_rate, samples_num):

    for s in range(5, 0, -1):
        print(s)
        time.sleep(1)
    print("Starting now!")

    myADC = ADC()
    myADC.addChannels([0], minRange=-10, maxRange=10)

    stop_event = Event()
    stopper = Thread(target=_wait_for_enter, args=(stop_event,), daemon=True)
    stopper.start()

    # Running lists
    time_stamps_sec = []
    averaged_voltages = []
    voltage_stds = []

    total_samples_read = 0
    print("Acquisition started — press ENTER to stop.\n")

    while not stop_event.is_set():

        block = np.asarray(myADC.sampleVoltages(samples_num, sample_rate)[0], dtype=float).reshape(-1)

        block_mean = float(block.mean())
        block_std  = float(block.std(ddof=1)) if block.size > 1 else 0.0

        mid_sample_index = total_samples_read + (samples_num / 2.0)
        mid_time_s = mid_sample_index / sample_rate

        time_stamps_sec.append(mid_time_s)
        averaged_voltages.append(block_mean)
        voltage_stds.append(block_std)

        total_samples_read += block.size

    df = pd.DataFrame({
        "time_s": np.array(time_stamps_sec),
        "ch0_mean_V": np.array(averaged_voltages),
        "ch0_std_V": np.array(voltage_stds),
    })
    return df

run_label = input("Enter run label (e.g. 'reference_lamp' or 'iodine_sample'): ").strip() or "run"
lambda_start_nm = float(input("Enter START wavelength from spectrometer display (nm): ").strip())

df = acquire_channel0_simple(sample_rate, samples_num)

elapsed_s = df["time_s"].iloc[-1] + (samples_num / sample_rate) / 2.0 if len(df) else 0.0
lambda_end_nm = float(input("Enter END wavelength from spectrometer display (nm): ").strip())

if elapsed_s > 0:
    df["lambda_nm_est"] = lambda_start_nm + (lambda_end_nm - lambda_start_nm) * (df["time_s"] / elapsed_s)
else:
    df["lambda_nm_est"] = np.nan

pd.DataFrame.to_csv(df, Path(f"ADC_{run_label}.csv"), index=False)

print(f"Points: {len(df)} | Duration: {elapsed_s:.2f} s | Start λ: {lambda_start_nm} nm | End λ: {lambda_end_nm} nm")
